In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pathlib import Path
from urllib.parse import urljoin, urlparse
import requests
import time

In [3]:
BASE_URL = "https://manga.in.ua"
DOWNLOAD_FOLDER = Path("data")

MANGAS = {
    # "simja-shpiguna": "https://manga.in.ua/chapters/65063-simja-shpiguna-tom-12-rozdil-73.html",
    "rab-elitnih-vijsk": "https://manga.in.ua/chapters/79869-rab-elitnih-vijsk-magichnogo-mista-tom-1-rozdil-1.html",
    "mij-najkraschij-drug-ziznavsja-meni-u-simuljatori-pobachen": "https://manga.in.ua/chapters/101207-mij-najkraschij-drug-ziznavsja-meni-u-simuljatori-pobachen-tom-1-rozdil-1.html",
    "mikadono-sanshimai-wa-angai-choroi": "https://manga.in.ua/mangas/budenst/106420-mikadono-sanshimai-wa-angai-choroi.html",
    "magichna-bitva-mojuro": "https://manga.in.ua/chapters/102102-mojuro-1.html",
    "hlopec-poverhom-vische": "https://manga.in.ua/chapters/28713-hlopec-poverhom-vische-tom-0-rozdil-1.html"
    }

In [4]:
# --- Налаштування Selenium ---
chrome_options = Options()
# chrome_options.add_argument("--headless")  # без GUI
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--log-level=3")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(5)

In [5]:
def download_chapter(driver: webdriver.Chrome, out_dir: Path):
    # 1️⃣ кнопка "Читати розділ" (без except)
    read_btns = driver.find_elements(
        By.CSS_SELECTOR, 'a[onclick*="LoadMangaImages"]'
    )
    if read_btns:
        read_btns[0].click()
        print("Кнопка 'Читати розділ' натиснута")
    else:
        print("Кнопки 'Читати розділ' немає — контент вже завантажений")

    # 2️⃣ чекаємо контейнер з картинками
    comics_div = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "comics"))
    )
    time.sleep(4)  # lazy-load

    # 3️⃣ дістаємо картинки
    images = comics_div.find_elements(By.CSS_SELECTOR, "img[data-src]")
    if not images:
        raise RuntimeError("Не знайдено картинок у розділі")

    # 4️⃣ завантажуємо
    out_dir.mkdir(parents=True, exist_ok=True)
    for idx, img in enumerate(images, start=1):
        img_url = urljoin(BASE_URL, img.get_attribute("data-src"))
        ext = Path(urlparse(img_url).path).suffix or ".jpg"
        filename = out_dir / f"{idx:03d}{ext}"
        filename.write_bytes(requests.get(img_url).content)

    print(f"Завантажено {len(images)} сторінок у {out_dir}")


In [6]:
for manga_name, start_url in MANGAS.items():
    print(f"\n===== МАНГА: {manga_name} =====")

    driver.get(start_url)
    time.sleep(3)

    while True:
        chapter_url = driver.current_url
        folder_name = chapter_url.split("/")[-1].replace(".html", "")
        out_folder = DOWNLOAD_FOLDER / manga_name / folder_name

        print(f"\n=== Розділ: {folder_name} ===")

        try:
            download_chapter(driver, out_folder)
        except Exception as e:
            print(f"❌ Помилка розділу: {e}")
            break

        # ▶ кнопка "Наступний"
        next_btns = driver.find_elements(By.ID, "fwdTop")
        if not next_btns:
            print("Кнопка 'Наступний' відсутня — кінець манги")
            break

        next_btn = next_btns[0]
        if "disabled" in next_btn.get_attribute("class"):
            print("Останній розділ манги")
            break

        next_btn.click()
        time.sleep(4)

    print(f"✅ Манга '{manga_name}' завершена")


===== МАНГА: rab-elitnih-vijsk =====

=== Розділ: 79869-rab-elitnih-vijsk-magichnogo-mista-tom-1-rozdil-1 ===
Кнопка 'Читати розділ' натиснута
Завантажено 59 сторінок у data/rab-elitnih-vijsk/79869-rab-elitnih-vijsk-magichnogo-mista-tom-1-rozdil-1

=== Розділ: 79870-rab-elitnih-vijsk-magichnogo-mista-tom-1-rozdil-2 ===
Кнопка 'Читати розділ' натиснута
❌ Помилка розділу: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=143.0.7499.193)
Stacktrace:
0   chromedriver                        0x0000000102a2fdfc cxxbridge1$str$ptr + 3031016
1   chromedriver                        0x0000000102a27cb8 cxxbridge1$str$ptr + 2997924
2   chromedriver                        0x0000000102522b90 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74192
3   chromedriver                        0x00000001024fc948 chromedriver + 149832
4   chromedriver                        0x0000000102591b38 _RNvCsgXDX2mvAJAg_7___rustc35___

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=143.0.7499.193)
Stacktrace:
0   chromedriver                        0x0000000102a2fdfc cxxbridge1$str$ptr + 3031016
1   chromedriver                        0x0000000102a27cb8 cxxbridge1$str$ptr + 2997924
2   chromedriver                        0x0000000102522b90 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74192
3   chromedriver                        0x00000001024fc948 chromedriver + 149832
4   chromedriver                        0x0000000102591b38 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 528760
5   chromedriver                        0x00000001025aa58c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 629708
6   chromedriver                        0x000000010255e22c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 317548
7   chromedriver                        